## CS431/631 Data Intensive Distributed Analytics
### Winter 2022 - Assignment 3
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** Clement Wu
* **ID:** 20775633

---
#### Overview
For this assignment, you will be using Python and Spark to perform some graph analysis, using a graph of the Gnutella server network.   In this graph, each node represents a server, and each (directed) edge represents a connection between servers in Gnutella's peer-to-peer network.  The input file for this assignment, `p2p-Gnutella08-adj.txt`, represents the graph as an adjacency list.   Each server (node) is identified by a unique number, and each line in the file gives the adjacency list for a single server.
For example, this line:
> 91	243	1923	2194

gives the adjacency list for server `91`.   It indicates that there are edges from server `91` to servers `243`, `1923`, and `2194`.    According to the Stanford Network Analysis Project, which collected these data, [the graph includes 6301 servers and 20777 edges](http://snap.stanford.edu/data/p2p-Gnutella08.html).

Run the following block to install Spark and download the input file.

In [ ]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
!pip install -q findspark
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/p2p-Gnutella08-adj.txt

and then create a `SparkContext`:

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

spark_conf = SparkConf()\
  .setAppName("YourTest")\
  .setMaster("local[*]")

sc = SparkContext.getOrCreate(spark_conf)

---
### Important

###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions. Do **NOT** write code outside of functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.

---
#### Question 1  (6/24 marks):

To get warmed up, you should first write Spark code to confirm or determine some basic properties of the Gnutella graph.  Write code in the provided functions that will return answers to the following questions, as specified in each function's documentation:
- How many nodes and edges are there in the graph?  (This should confirm the numbers given above.)
- How many nodes of each outdegree are there? That is, how many nodes have no outgoing edges, how many have one outgoing edge, how many have two outgoing edges, and so on?
- How many nodes of each indegree are there?

You should use Spark to answer these questions.   Do *not* load the entire graph into your Python driver program.

In [219]:
def num_nodes_edges():
    """Returns a tuple (num_nodes, num_edges)"""
    textFile = sc.textFile('p2p-Gnutella08-adj.txt')
    counts = textFile.count()  # count lines of the file
    edge_counts = textFile.map(lambda line : len(line.split("\t"))-1).reduce(lambda a,b : a+b) # count every adjacent nodes and take the sum
    return (counts, edge_counts)

    
    
def out_counts():
    """Returns a dictionary where the keys are the outdegrees, and the 
    values are the number of nodes of the corresponding outdegree """
    textFile = sc.textFile('p2p-Gnutella08-adj.txt')
    outdegrees = textFile.map(lambda line : line.split("\t")).map(lambda x : (len(x)-1, 1)).reduceByKey(lambda x, y : x + y) # count every adjacent nodes, reverse key-value pairs to get accumulate frequency
    return outdegrees.collectAsMap()
    


def in_counts():
    """Returns a dictionary where the keys are the indegrees, and the 
    values are the number of nodes of the corresponding indegree """
    textFile = sc.textFile('p2p-Gnutella08-adj.txt')
    indegrees = textFile.map(lambda line : line.split("\t")).map(lambda x : x[1:]).flatMap(lambda x : x)\
    .map(lambda x : (x, 1)).reduceByKey(lambda x,y:x+y)  # get number of in-degrees of every node
    counts = indegrees.map(lambda x : (int(x[1]), 1)).reduceByKey(lambda x,y:x+y) # count in-degree frequency based on number of in-degrees of every node
    return counts.collectAsMap()




---
Your main objective for this assignment is to perform *single source personalized page rank* over the Gnutella graph.  To get started, you should make sure that you have a clear understanding of ordinary (i.e., non-personalized) page rank.  Read the description of page rank in Section 5.3 of [the course textbook](http://mapreduce.cc/).   Personalized page rank is like ordinary page rank except:
- One node in the graph is designated as the *source* node. Personalized page rank is performed with respect to that source node.
- Personalized page rank is initialized by assigning all probability mass to the source node, and none to the other nodes. In contrast, ordinary page rank is initialized by giving all nodes the same probability mass.
- Whenever personalized page rank makes a random jump, it jumps back to the source node. In contrast, ordinary page rank may jump to any node.
- In personalized page rank, all probability mass lost dangling nodes is put back into the source nodes.  In ordinary page rank, lost mass is distributed evenly over all nodes.

#### Question 2  (10/24 marks):

Your task is to write a Spark program to perform personalized page rank over the Gnutella graph for a specified number of iterations, and of course a specific node. The function you will implement takes three input values:
- source node id (a non-negative integer)
- iteration count (a positive integer)
- random jump factor value (a float between 0 and 1) - This is 1-B as introduced in the lecture.

The function should perform personalized page rank, with respect to the specified source node, over the Gnutella graph, for the specified number of iterations, using Spark.
The output of your function should be a list of the 10 nodes with the highest personalized page rank with respect to the given source. For each of the 10 nodes, return the node's id and page rank value as a tuple. The list returned by the function should therefore look something like this: `[(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]`

In [224]:
from operator import add
def personalized_page_rank(source_node_id, num_iterations, jump_factor):
    """Returns a list of the 10 nodes with the highest page rank value along with their value, as tuples
    [(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]"""


    def initialization(node):
      """Initialize the probability of node by setting 1 to source node and 0 to others"""
      return (node, 1) if node==source_node_id else (node, 0)
    
    def distribute(pair):
      """input pair of (adjacent_node_list, node_probability), and distribute the probability evenly to all adjacent nodes in the list"""
      return [(s, pair[1]/len(pair[0])) for s in pair[0]]
    
    def update(node, p, jump_prob):
      """update the node probability. For source node, we multiply its probability by (1-jump_factor), then add other nodes jump probability.
      For other nodes, simply multiply by (1-jump_factor)"""
      if node==source_node_id:
        return (1-jump_factor)*p + jump_prob
      return (1-jump_factor)*p
    
    def strToInt(alist):
      """change all strings in a list to int"""
      return list(map(int, alist))

    adj_matrix = sc.textFile('p2p-Gnutella08-adj.txt').map(lambda x : x.split("\t")).map(lambda x : (int(x[0]), strToInt(x[1:]))).cache() # intialize adjacent list in a form (node, list of adjacent nodes)

    node_list = adj_matrix.map(lambda x : initialization(x[0])) #initialize node probability in a form (node, p)
    
    for i in range(num_iterations):
      map_reduce = adj_matrix.join(node_list).map(lambda x : distribute(x[1])).flatMap(lambda x : x).reduceByKey(lambda x, y: x + y).cache() # distrubute probability to adjacent node and sum up partial probability for all nodes
      jump_prob = 1 - (1-jump_factor)*(map_reduce.map(lambda x : x[1]).reduce(lambda x, y : x + y)) # calculate total jump probability from other nodes
      node_list = map_reduce.map(lambda x : (x[0], update(x[0], x[1], jump_prob))).cache() # update all nodes with new probability
              
    return node_list.sortBy(lambda x : x[1], ascending=False).take(10) # find top 10 nodes with highest probability

    



In [225]:
personalized_page_rank(2794,10,0.05)

[(2794, 0.36541559072338653),
 (513, 0.036649507904412876),
 (2001, 0.036594127676179304),
 (2292, 0.03646168485025542),
 (2615, 0.03617013461969299),
 (1509, 0.03616197848386497),
 (3215, 0.036146086303539875),
 (1042, 0.03614599491394118),
 (4306, 0.036144776088395676),
 (3907, 0.03614475784022657)]

---
#### Question 3  (4/24 marks):

For the previous question, you implemented personalized page rank that ran for a specified number of iterations.  However, it is also common to write iterative algorithms that run until some specified termination condition is reached.
For example, for page rank, suppose the $p_i(x)$ represents the probability mass assigned to node $x$ after the $i$th iteration of the algorithm.  ($p_0(x)$ is the initial probability mass of node $x$.)   We define the change of $x$'s probability mass on the $i$th iteration as $\lvert p_i(x)-p_{i-1}(x) \rvert$.   Then, we can iterate personalized page rank until the maximum (over all nodes) change is less than a specified threshold, i.e, until all nodes' page ranks have converged.

For this question, modify your personalized page rank implementation from Question 2 so that it iterates until the 
maximum node change is less than $\frac{0.5}{N}$, where $N$ represents the number of nodes in the graph.
This version of the function should take only two inputs: the source node id and the random jump factor.

In [217]:
def personalized_page_rank_stopping_criterion(source_node_id, jump_factor):
    """Returns a list of the 10 nodes with the highest page rank value along with their value, as tuples
    [(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]"""

    def initialization(node):
      """Initialize the probability of node by setting 1 to source node and 0 to others"""
      return (node, 1) if node==source_node_id else (node, 0)
    
    def distribute(pair):
      """input pair of (adjacent_node_list, node_probability), and distribute the probability evenly to all adjacent nodes in the list"""
      return [(s, pair[1]/len(pair[0])) for s in pair[0]]
    
    def update(node, p, jump_prob):
      """update the node probability. For source node, we multiply its probability by (1-jump_factor), then add other nodes jump probability.
      For other nodes, simply multiply by (1-jump_factor)"""
      if node==source_node_id:
        return (1-jump_factor)*p + jump_prob
      return (1-jump_factor)*p
    
    def strToInt(alist):
      """change all strings in a list to int"""
      return list(map(int, alist))

    adj_matrix = sc.textFile('p2p-Gnutella08-adj.txt').map(lambda x : x.split("\t")).map(lambda x : (int(x[0]), strToInt(x[1:]))).cache() # intialize adjacent list in a form (node, list of adjacent nodes)

    node_list = adj_matrix.map(lambda x : initialization(x[0])) #initialize node probability in a form (node, p)

    old_node_list = node_list # record node with old probability

    num_node = 6301

    maximum = 1
    
    while maximum >= 0.5/num_node:
      old_node_list = node_list # update old node probability list
      map_reduce = adj_matrix.join(node_list).map(lambda x : distribute(x[1])).flatMap(lambda x : x).reduceByKey(lambda x, y: x + y).cache() # distrubute probability to adjacent node and sum up partial probability for all nodes
      jump_prob = 1 - (1-jump_factor)*map_reduce.map(lambda x : x[1]).reduce(lambda x, y : x + y) # calculate total jump probability from other nodes
      node_list = map_reduce.map(lambda x : (x[0], update(x[0], x[1], jump_prob))).cache() # update all nodes with new probability
      maximum = node_list.join(old_node_list).map(lambda x : abs(x[1][0]-x[1][1])).reduce(max) # update maximum by taking the absolute value of the difference of new and old prob
              
    return node_list.sortBy(lambda x : x[1], ascending=False).take(10) # find top 10 nodes with highest probability


In [218]:
personalized_page_rank_stopping_criterion(2750, 0.1)

[(2750, 0.30220394870719725),
 (7, 0.0346432053832001),
 (174, 0.033601317145302304),
 (128, 0.029311063948404725),
 (856, 0.028554469248364647),
 (717, 0.02845676942362639),
 (368, 0.028445547796930898),
 (424, 0.028153777250628887),
 (125, 0.027985045265408154),
 (238, 0.027689556656325602)]

---
#### Question 4  (4/24 marks):

Spark provides the ability to *cache* RDDs.   This is useful when an RDD will be used more than once.   Instead of computing the same RDD multiple times, it can be computed once, cached, and then re-used from the cache.   Read about caching in the Spark textbook, or in the [persistence section of the Spark RDD programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence).   Caching can be particularly effective for iterative Spark applications, like those you are writing for this assignment.

For this question, go back to the code that you wrote to answer Question 3, and add caching.   (Caching will not affect the functionality of your code, i.e., what it computes.   It should only affect performance.)   Don't worry about different persistence levels.   Just use `cache()`, which caches RDDs at the default persistence level.

In addition to adding `cache()` calls in your code, use the text cell below to briefly explain how you decided which RDDs to cache. 

If you were not able to finish Question 3, add caching annotations to your solution for Question 2 instead.

---
#### Your answer to Question 4:

I add cache() to adj_matrix because it is used repeartly in the while loop. Also in the while loop, I apply cache() to map_reduce because there are multiple lines after it have operations related to it. I also apply cache() to node_list because it was repeatly updated in every iteration, and map_reduce rdd uses it to distribute probabilities.




---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.